![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

# Ingeniería de características

## Suavizado de series de tiempo

#### [Julio Waissman Vilanova](julio.waissman@unison.mx)

Vamos a ver diferentes tipos y formas de suavisar curvas. Para esto, vamos a utilizar como serie de tiempo la serie de casos confirmados por COVID-19 recabados y mantenidos por [Luis Armando Moreno](http://www.luisarmandomoreno.com), los cuales se pueden descargar [aquí](https://onedrive.live.com/download.aspx?resid=5ADDF6870413EAC9!40221&authkey=!AHWUE_EQfhvGRm4).


In [2]:
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objects as go
import plotly.express as px


confirmados = pd.read_csv(
    "Casosdiarios.csv", 
    engine="python",
    parse_dates=['Fecha']
)[['Fecha', 'CASOS']] \
    .groupby("Fecha") \
    .sum() \
    .diff() + 1


fig = px.scatter(
    confirmados,
    y="CASOS"
).update_layout(
    title='Casos confirmados de COVID-10 en Sonora',
    hovermode="x"
).show()

### Suavizado por medias moviles

El suavizado por media movil utiliza una ventana de tiempo en los datos para suavizar. La ventana de tiempo debe de tener sentido para los datos, pero se puede jugar con ella. 

Para esto se usa el método `rolling` el cual se puede usar con otros tipos de funciones.


In [3]:
confirmados["ma 3"] = confirmados.CASOS.rolling(window=3, center=True).mean()
confirmados["ma 7"] = confirmados.CASOS.rolling(window=7, center=True).mean()
confirmados["ma 14"] = confirmados.CASOS.rolling(window=14, center=True).mean()

fig = go.Figure(
).add_scatter(
    x=confirmados.index,
    y=confirmados["CASOS"],
    mode='markers',
    name="Real"
).add_scatter(
    x=confirmados.index,
    y=confirmados["ma 3"],
    hovertemplate="%{y:.1f}",
    name="MA 3 días"
).add_scatter(
    x=confirmados.index,    
    y=confirmados["ma 7"],
    hovertemplate="%{y:.1f}",
    name="MA 7 días"
).add_scatter(
    x=confirmados.index,
    y=confirmados["ma 14"],
    hovertemplate="%{y:.1f}",
    name="MA 14 días"
).update_layout(
    title='Casos confirmados de COVID-10 en Sonora',
    hovermode="x unified"
).show()

### Medianas moviles exponenciales


In [9]:
confirmados["mm 3"] = confirmados.CASOS.rolling(window=3, center=True).median()
confirmados["mm 7"] = confirmados.CASOS.rolling(window=7, center=True).median()
confirmados["mm 14"] = confirmados.CASOS.rolling(window=14, center=True).median()

fig = go.Figure(
).add_scatter(
    x=confirmados.index,
    y=confirmados["CASOS"],
    mode='markers',
    name="Real"
).add_scatter(
    x=confirmados.index,
    y=confirmados["mm 3"],
    hovertemplate="%{y:.1f}",
    name="MM 3 días"
).add_scatter(
    x=confirmados.index,    
    y=confirmados["mm 7"],
    hovertemplate="%{y:.1f}",
    name="MM 7 días"
).add_scatter(
    x=confirmados.index,
    y=confirmados["mm 14"],
    hovertemplate="%{y:.1f}",
    name="MM 14 días"
).update_layout(
    title='Casos confirmados de COVID-10 en Sonora',
    hovermode="x"
).show()

### Medias moviles exponenciales

In [11]:
confirmados["ewm 3"] = confirmados.CASOS.ewm(span=3).mean()
confirmados["ewm 7"] = confirmados.CASOS.ewm(span=7).mean()
confirmados["ewm 14"] = confirmados.CASOS.ewm(span=14).mean()

fig = go.Figure(
).add_scatter(
    x=confirmados.index,
    y=confirmados["CASOS"],
    mode='markers',
    name="Real"
).add_scatter(
    x=confirmados.index,
    y=confirmados["ewm 3"],
    hovertemplate="%{y:.1f}",
    name="EWM 3 días"
).add_scatter(
    x=confirmados.index,    
    y=confirmados["ewm 7"],
    hovertemplate="%{y:.1f}",
    name="EWM 7 días"
).add_scatter(
    x=confirmados.index,
    y=confirmados["ewm 14"],
    hovertemplate="%{y:.1f}",
    name="EWM 14 días"
).update_layout(
    title='Casos confirmados de COVID-10 en Sonora'
).show()

### LOWESS


In [12]:
lowess = sm.nonparametric.lowess

l1 = lowess(confirmados.CASOS, confirmados.index, frac=1/5)
l2 = lowess(confirmados.CASOS, confirmados.index, frac=1/10)
l3 = lowess(confirmados.CASOS, confirmados.index, frac=1/20)

fig = go.Figure(
).add_scatter(
    x=confirmados.index,
    y=confirmados["CASOS"],
    mode='markers',
    name="Real"
).add_scatter(
    x=confirmados.index[1:],
    y=l1[:,1],
    hovertemplate="%{y:.1f}",
    name="LOWESS 1/5"
).add_scatter(
    x=confirmados.index[1:],    
    y=l2[:,1],
    hovertemplate="%{y:.1f}",
    name="LOWESS 1/10"
).add_scatter(
    x=confirmados.index[1:],
    y=l3[:,1],
    hovertemplate="%{y:.1f}",
    name="LOWESS 1/20"
).update_layout(
    title='Casos confirmados de COVID-10 en Sonora',
    hovermode="x"
).show()



In [14]:
fig = go.Figure(
).add_scatter(
    x=confirmados.index,
    y=confirmados["CASOS"],
    mode='markers',
    name="Real"
).add_scatter(
    x=confirmados.index[1:],
    y=l2[:,0],
    hovertemplate="%{y:.1f}",
    name="LOWESS 1/5"
).add_scatter(
    x=confirmados.index[1:],    
    y=l2[:,1],
    hovertemplate="%{y:.1f}",
    name="LOWESS 1/10"
).update_layout(
    title='Casos confirmados de COVID-10 en Sonora',
    hovermode="x"
).show()




In [13]:
fig = go.Figure(
).add_scatter(
    x=confirmados.index,
    y=confirmados["CASOS"],
    mode='markers',
    name="Real"
).add_scatter(
    x=confirmados.index,
    y=confirmados["ma 7"],
    hovertemplate="%{y:.1f}",
    name="MA 7 días"
).add_scatter(
    x=confirmados.index,    
    y=confirmados["mm 7"],
    hovertemplate="%{y:.1f}",
    name="MM 7 días"
).add_scatter(
    x=confirmados.index,
    y=confirmados["ewm 7"],
    hovertemplate="%{y:.1f}",
    name="EWM 7 días"
).add_scatter(
    x=confirmados.index[1:],
    y=l2[:,1],
    hovertemplate="%{y:.1f}",
    name="LOWESS 1/10"
).update_layout(
    title='Casos confirmados de COVID-10 en Sonora',
    hovermode="x"
).show()


